# "Fallecidos DEIS por comuna"
> Fallecidos DEIS por comuna.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import numpy as np
import pandas as pd
import altair as alt
from IPython.display import display, Markdown, display_html, HTML

In [5]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(19)

In [6]:
#hide_input
display(Markdown(f"Última actualización: {update_date.strftime('%d/%m/%Y')}."))

Última actualización: 29/06/2020.

In [3]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/alonsosilvaallende/COVID-19/master/data/DEIS_causa_COVID.csv")

# Fallecidos DEIS por comuna

In [4]:
#hide
data_raw = data_raw.drop(columns="año")

In [8]:
#hide
data_RM = data_raw.query("region=='Metropolitana de Santiago'")

In [10]:
#hide
confirmed_deaths = pd.Series(dtype="float64")
suspected_deaths = pd.Series(dtype='float64')
for comuna in data_RM["comuna"].unique():
    confirmed_deaths[f"{comuna}"] = len(data_raw[(data_raw["código_detalle"] == 'U07.1') & (data_raw["comuna"] == f'{comuna}')])
    suspected_deaths[f"{comuna}"] = len(data_raw[(data_raw["código_detalle"] == 'U07.2') & (data_raw["comuna"] == f'{comuna}')])

In [11]:
#hide
deaths = pd.DataFrame()
deaths["Comuna"] = data_RM["comuna"].unique()
deaths = deaths.set_index("Comuna")

In [12]:
#hide
deaths["COVID Confirmado"] = confirmed_deaths
deaths["COVID Sospechoso"] = suspected_deaths

In [13]:
#hide
deaths_long_format = deaths.reset_index().melt("Comuna", var_name="Causa", value_name="Fallecidos")

In [25]:
#hide_input
bars = alt.Chart(deaths_long_format).mark_bar(opacity=0.7, size=25).encode(
    x = alt.X('Fallecidos:Q', stack='zero'),
    y = alt.Y('Comuna', sort='-x'),
    tooltip = ['Comuna', 'Fallecidos'],
    color = alt.Color('Causa', sort=['Confirmado','Sospechoso']),
    order = alt.Order('Causa')
)

text = alt.Chart(deaths_long_format).mark_text(dx=-11, dy=0, color="#636363").encode(
     x=alt.X('Fallecidos:Q', stack='zero'),
     y=alt.Y('Comuna', sort='-x'),
     color=alt.Color('Causa', sort=['Confirmado','Sospechoso']),
     order = alt.Order("Causa"),
     text=alt.Text("Fallecidos", format=',.0f')
)
(bars+text).properties(
    title = f"Total fallecidos DEIS por comuna de la R.M. al {pd.to_datetime('2020-06-28').strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
)

alt.LayerChart(...)